__INTERNE RED WINES DATASET__

__EXTERNE WEER DATASET__

In [280]:
# importeer nodige libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [281]:
# lees de externe dataset in
weather_conditions = pd.read_csv("weatherdata_lisbon.csv", delimiter=";", encoding='iso-8859-1')
weather_conditions.head()

,STN---,YEARMODA,TEMP,DEWP,VISIB,Unnamed: 5,WDSP,Unnamed: 7,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT
0,85360,19900101,"55,1","50,5","6,3",24,"5,8",24,13,"999,9",59.0*,44.6*,"99,99","999,9",10000
1,85360,19900102,"50,2","47,7",5,24,"5,8",24,21,"999,9",59.0*,42.8*,"99,99","999,9",110010
2,85360,19900103,"49,5","46,6","6,6",24,"4,8",24,"9,9",20,55.4*,44.6*,"99,99","999,9",10000
3,85360,19900104,"52,1","50,7",5,24,"1,7",24,6,"999,9",57.2*,46.4*,"99,99","999,9",10000
4,85360,19900105,"54,2",54,"1,2",22,1,22,"4,1","999,9",55.4*,53.6*,"99,99","999,9",110000


In [282]:
weather_conditions.drop(['STN---','Unnamed: 5','Unnamed: 7','GUST','PRCP','SNDP','VISIB'], axis=1,inplace=True)
weather_conditions.dropna(inplace=True)
weather_conditions.rename(columns={'YEARMODA': 'Year','DEWP':'Dewpoint Temperature','TEMP': 'Temperature','MAX':'Max Temperature','MIN':'Min Temperature','WDSP':'Windspeed','MXSPD':'Max Windspeed'},inplace=True)

In [283]:
# cleanup and transform data types
# also convert fahrenheit to celsius
weather_conditions['Max Temperature'] = (weather_conditions['Max Temperature'].str.replace(',','.').str.rstrip("*").astype(float) - 32) / 1.8
weather_conditions['Min Temperature'] = (weather_conditions['Min Temperature'].str.replace(',','.').str.rstrip("*").astype(float) - 32) / 1.8
weather_conditions['Temperature'] = (weather_conditions['Temperature'].str.replace(',','.').astype(float) - 32) / 1.8
weather_conditions['Dewpoint Temperature'] = (weather_conditions['Dewpoint Temperature'].str.replace(',','.').astype(float) - 32) / 1.8
weather_conditions['Windspeed'] = weather_conditions['Windspeed'].str.replace(',','.').astype(float)
weather_conditions['Max Windspeed'] = weather_conditions['Max Windspeed'].str.replace(',','.').astype(float)
weather_conditions['Year']= pd.to_datetime(weather_conditions['Year'].astype(str), format='%Y-%m-%d')

In [284]:
# FRSHTT = Frost, Rain, Snow, Hail, Thunder, Thornado
weather_conditions['FRSHTT'] = weather_conditions['FRSHTT'].apply(lambda x: '{0:0>6}'.format(x))
weather_conditions['Frost'] = weather_conditions['FRSHTT'].str[0].astype(float)
weather_conditions['Rain'] = weather_conditions['FRSHTT'].str[1].astype(float)
weather_conditions['Snow'] = weather_conditions['FRSHTT'].str[2].astype(float)
weather_conditions['Hail'] = weather_conditions['FRSHTT'].str[3].astype(float)
weather_conditions['Thunder'] = weather_conditions['FRSHTT'].str[4].astype(float)
weather_conditions['Tornado'] = weather_conditions['FRSHTT'].str[5].astype(float)
weather_conditions.drop('FRSHTT',axis=1,inplace=True)

In [285]:
weather_conditions = weather_conditions[weather_conditions['Min Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Max Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Windspeed'] < 50]
weather_conditions = weather_conditions[weather_conditions['Max Windspeed'] < 50]
weather_conditions = weather_conditions[weather_conditions['Dewpoint Temperature'] < 100]

In [286]:
weather_conditions['Count'] = 1
weather_conditions_peryear = weather_conditions.groupby(temperatures['Year'].map(lambda x: x.year)).sum()
weather_conditions_peryear = weather_conditions_peryear[['Temperature','Dewpoint Temperature','Windspeed','Max Windspeed','Max Temperature','Min Temperature','Frost','Rain','Snow','Hail','Thunder','Tornado']].div(weather_conditions_peryear['Count'], axis=0)
weather_conditions_peryear.drop(pd.Int64Index([2018]), inplace=True)